In [1]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets
import torch
import numpy
import matplotlib.pyplot as plt
import datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import argparse
import pdb

from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments

print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
args_dict = {
    "train_dataset": "/content/drive/MyDrive/OMSCS_DL/HP/data/train.jsonl",
    "val_dataset": "/content/drive/MyDrive/OMSCS_DL/HP/data/dev.jsonl",
    "test_dataset": "/content/drive/MyDrive/OMSCS_DL/HP/data/test.jsonl",
    "model": "roberta-base",
    "output_dir": "/content/drive/MyDrive/OMSCS_DL/HP/output/step_three/mlm/",
    "adapter_name": "HP_adapter"
}

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-m2pl6dy9
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-m2pl6dy9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 9.0MB/s 
     |████████████████████████████████| 3.3MB 26.3MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0-cp37-none-any.whl size=2097542 sha256=a38888e1ca2eb0d72c27f1134c65a562d5694f40e3d540b14e74331c00de34f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-eqks8wpc/wheels/b0/56/c9/5bf1c51cd513412090ad751ab10fc025210176bf0a82dd8af3
Successfully built adapter-transformers
     |████████████████████████████████| 225kB 8.5MB/s 
     |████████████████████████████████| 112kB 9.2MB/s 
     |████████████████████████████████| 245kB 13.7MB/s 
1
Tesla P100-PCIE-16GB


In [2]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')
train_data = datasets.load_dataset("json", data_files=args_dict["train_dataset"])["train"]
#not a typo: load_dataset loads this as training data
val_data = datasets.load_dataset("json", data_files=args_dict["val_dataset"])["train"] 


def tokenization(batched_text):
    tokenized_batch = tokenizer(batched_text['text'], padding=True, truncation=True, return_special_tokens_mask=True)
    return tokenized_batch

train_data = train_data.map(tokenization, batched=True, batch_size=len(train_data), remove_columns=["text", "label"])
train_data.set_format('torch', columns=['input_ids'])


val_data = val_data.map(tokenization, batched=True, batch_size=len(train_data), remove_columns=["text", "label"])
val_data .set_format('torch', columns=['input_ids'])

#static masking. Possible TODO: use dynamic masking
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

adapter_name = "mlm_HP_pretrain"
model.add_adapter(adapter_name)
model.train_adapter(adapter_name)

Using custom data configuration default-fe5a4820a26adf77


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-fe5a4820a26adf77/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-034d47800bfe2ca3


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-034d47800bfe2ca3/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [3]:
training_args = TrainingArguments(
    num_train_epochs=110,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    #lr_scheduler_type="constant",        
    adam_epsilon=1e-6,
    adam_beta1=0.9,
    adam_beta2=0.98,
    weight_decay=0.01,
    warmup_ratio=0.06,
    fp16=True,
    eval_accumulation_steps=1,
    save_steps=5000,
    save_total_limit=2,
    overwrite_output_dir=True,
    remove_unused_columns=False,
    seed=3,
    output_dir=args_dict["output_dir"],
    evaluation_strategy = "epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=val_data
)

In [4]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,5.148083,1.827700,35.016000
2,No log,4.872166,1.838200,34.817000
3,No log,4.734595,1.837800,34.824000
4,No log,4.483617,1.832800,34.920000
5,No log,4.211255,1.826900,35.033000
6,No log,4.167366,1.824000,35.089000
7,No log,4.030429,1.832900,34.917000
8,No log,3.649988,1.822400,35.118000
9,No log,3.710781,1.838100,34.818000
10,No log,3.437675,1.823200,35.104000


TrainOutput(global_step=3630, training_loss=2.7353028184454633, metrics={'train_runtime': 3691.2198, 'train_samples_per_second': 0.983, 'total_flos': 2.189905829793792e+16, 'epoch': 110.0, 'init_mem_cpu_alloc_delta': 1643278336, 'init_mem_gpu_alloc_delta': 503669760, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 18161664, 'train_mem_gpu_alloc_delta': 18505728, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 14258256896})

In [5]:
 model.save_adapter(args_dict["output_dir"] + adapter_name, adapter_name)